In [46]:
from ngsolve import *
from netgen.geom2d import *
from netgen.occ import *
from ngsolve.webgui import Draw

# ------------------------------------------------------------
# Parameter
# ------------------------------------------------------------
H, L = 4.0, 28.0
cx, cy, R = 7.0, 0.0, 0.5

nu = 1e-3       #viscosity

# ------------------------------------------------------------
# Geometrie
# Rectangle(width, height) liegt standardmäßig in [0,L]×[0,H]
# → daher zuerst erzeugen, dann verschieben
# ------------------------------------------------------------
rect = MoveTo(0,-H/2).Rectangle(L, H).Face()

rect.edges.Min(X).name = "inlet"
rect.edges.Max(X).name = "outlet"
rect.edges.Min(Y).name = "walls"
rect.edges.Max(Y).name = "walls"

cyl = Circle((cx,cy), R).Face()
cyl.edges.name = "obstacle"

shape = rect - cyl
#Draw(shape)

# ------------------------------------------------------------
# Mesh
# ------------------------------------------------------------
mesh = Mesh(OCCGeometry(shape,dim=2).GenerateMesh(maxh=0.4))
mesh.Refine()
Draw(mesh);

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [47]:
# ------------------------------------------------------------
# FE-Räume (Taylor–Hood)
# ------------------------------------------------------------
V = VectorH1(mesh, order=2, dirichlet="inlet|walls|obstacle")
Q = H1(mesh, order=1)   # Variante C: integral(p)=0

X = FESpace([V, Q], constraints=[])
(u, p) = X.TrialFunction()
(v, q) = X.TestFunction()

gfu = GridFunction(X)
gfu_u, gfu_p = gfu.components


In [48]:
# ------------------------------------------------------------
# Inlet-Profil (parabolisch auf [-H/2, H/2])            -> Randbedingungen bei Inlet:  u = uin bei "inlet"
# ------------------------------------------------------------
Umax = 0.1
uin_x = Umax*(1 - (2*y/H)**2)   # Maximum bei y=0
uin = CoefficientFunction((uin_x, 0))

gfu_u.Set(uin, definedon=mesh.Boundaries("inlet"))


In [ ]:
# ------------------------------------------------------------
# Stokes: schwache Form
# ------------------------------------------------------------

#für stabilität
alpha = 1e-10
gamma = 1e-2


a = BilinearForm(X)
a += 2*nu*InnerProduct(Grad(u), Grad(v)) * dx         #TODO Sym weglassen
a += gamma * div(u) * div(v) * dx
a += -div(v)*p * dx
a += -div(u)*q * dx
a += alpha * p*q * dx

L = LinearForm(X)   # keine Volumenkräfte

a.Assemble()
L.Assemble()

# ------------------------------------------------------------
# Lösen
# ------------------------------------------------------------

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = L.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

Draw (gfu.components[0], mesh);
# ------------------------------------------------------------
# Visualisierung
# ------------------------------------------------------------
Draw(gfu_u, mesh, "velocity")
Draw(gfu_p, mesh, "pressure")


# Stokes hier fertig gelöst -> jetzt gehts an NavierStokes


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene